In [1]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../src')


import torch
import torch_geometric
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

from torch_geometric.datasets import TUDataset
from preprocessing import data_transformation
from similarity import calculate_similarity_matrix

from model import GCN

In [2]:
dataset = TUDataset(root="/", name="MUTAG")

## Preprocessing

#### Split: Train test validation

```train_dataset```: for training model<br/>
```val_dataset```: evaluate model for hyperparameter tunning<br/>
```test_dataset```: testing model after complete training<br/>

In [3]:
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data

In [18]:
tr, ts, vl = 0.8, 0.1, 0.1
dslen = len(dataset)
tri = round(tr*dslen)
tsi = round((tr+ts)*dslen)
train_dataset = dataset[:tri]
test_dataset = dataset[tri:tsi]
val_dataset = dataset[tsi:]

In [26]:
dataset.y

tensor([1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
        0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
        1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
        1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0])

In [32]:
print(len(train_dataset))
train_dataset.y

150


tensor([1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
        0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
        1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
        1, 1, 0, 1, 1, 0])

In [24]:
len(test_dataset)
test_dataset.y

tensor([1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1])

In [31]:
len(val_dataset)
val_dataset.y

tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0])

#### Batching

In [37]:
# paper 128
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [42]:
print('train loader')
for data in train_loader:
    print(data.y)
    
print('val loader')
for data in val_loader:
    print(data.y)
    
print('test loader')
for data in test_loader:
    print(data.y)

train loader
tensor([1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1])
tensor([0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
        0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1])
tensor([0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0])
val loader
tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0])
test loader
tensor([1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1])


## Building Model

In [43]:
from torch_geometric.nn import GCNConv
from torch.nn import Linear

from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import global_add_pool

### Base model

In [245]:
class Base(torch.nn.Module):
    # merging type: o --> complement only, s --> substraction, c --> concatenation
    def __init__(self, dataset, hidden_channels):
        super(Base, self).__init__()
        
        # weight seed
        torch.manual_seed(42)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        # classification layer
        
        self.lin = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # Embed original
        embedding = self.conv1(x, edge_index)
        embedding = embedding.relu()
        embedding = self.conv2(embedding, edge_index)
        # subgraph_embedding = subgraph_embedding.relu()
        
        embedding = global_mean_pool(embedding, batch)
        h = self.lin(embedding)
        h = h.relu()
        h = self.lin2(h)
        
        return embedding, h

In [246]:
base = Base(dataset, 64)
base

Base(
  (conv1): GCNConv(7, 64)
  (conv2): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=64, bias=True)
  (lin2): Linear(in_features=64, out_features=2, bias=True)
)

In [267]:
def train_base(model, loader):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    
    model.train()
    
    for data in loader:
        emb, h = model(data.x, data.edge_index, data.batch)
        loss = criterion(h, data.y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(h[0])
    print(loss)

@torch.no_grad()
def test_base(model, loader):
    model.eval()
    correct = 0
    for data in loader:
        emb, h = model(data.x, data.edge_index, data.batch)
        pred = h.argmax(dim=1)
        correct += int((pred == data.y).sum())
base = Base(dataset, 64)
base
train_base(base, train_loader)

tensor([ 0.0402, -0.0619], grad_fn=<SelectBackward0>)
tensor([-0.0620,  0.1474], grad_fn=<SelectBackward0>)
tensor([-0.1951,  0.3425], grad_fn=<SelectBackward0>)
tensor(0.8020, grad_fn=<NllLossBackward0>)
